
#  SQL Project — Book Reading Platform Analysis

## Introduction

The COVID-19 pandemic dramatically changed people’s daily lives. With restrictions on social activities, many individuals began spending more of their free time at home — and one of the most popular hobbies that emerged was reading.  
This shift in behavior drew the attention of several startups that started building new **book-lover applications** to meet the increasing demand for digital reading platforms.

As a data analyst, you have received a **database from one of these competitor services**, which contains data on books, authors, publishers, user ratings, and text reviews.  
Your task is to explore this database and extract insights that can help design a new, competitive product for book enthusiasts.

---

## Project Objectives

The main objectives of this analysis are:

1. **Count the number of books** published after **January 1st, 2000**.  
2. **Find the number of reviews** and the **average rating** for each book.  
3. **Identify the publisher** that released the **largest number of books with more than 50 pages** (to exclude short pamphlets).  
4. **Determine the author** with the **highest average book rating**, considering only books with at least **50 user ratings**.  
5. **Calculate the average number of reviews per user** among those who have rated **more than 50 books**.

---

## Database Structure

| Table | Description | Key Fields |
|--------|--------------|-------------|
| **books** | Information about books | `book_id`, `author_id`, `title`, `num_pages`, `publication_date`, `publisher_id` |
| **authors** | Information about authors | `author_id`, `author` |
| **publishers** | Information about publishing houses | `publisher_id`, `publisher` |
| **ratings** | Users’ numeric ratings for books | `rating_id`, `book_id`, `username`, `rating` |
| **reviews** | Users’ written reviews for books | `review_id`, `book_id`, `username`, `text` |

---

## Next Steps

1. Connect to the SQL database using **SQLAlchemy** and **Pandas**.  
2. Explore each table (`SELECT * FROM table LIMIT 5`) to understand its structure.  
3. Write and execute **SQL queries** to answer each business question.  
4. Summarize findings and insights for each task.

---

 **Goal:**  
By the end of this project, you will have a clear understanding of how users interact with books (ratings and reviews), which authors and publishers stand out, and which behaviors can inspire the development of a successful new book platform.


In [19]:
# importar bibliotecas
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'practicum_student', # nome de usuário
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # senha
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # porta de conexão
'db': 'data-analyst-final-project-db'} # o nome do banco de dados
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [33]:
# Definindo função para consultas SQL
# Não se esqueça das funções! Eles podem fazer a sua vida e a execução de consultas bem mais fácil.
def sql_results(query):
    #display(pd.io.sql.read_sql(query, con = engine))
    return pd.io.sql.read_sql(query, con = engine)

In [ ]:
# Estudando as tabelas
books_df = sql_results("SELECT * FROM books")
authors_df = sql_results("SELECT * FROM authors")
publishers_df = sql_results("SELECT * FROM publishers")
ratings_df = sql_results("SELECT * FROM ratings")
reviews_df = sql_results("SELECT * FROM reviews")

In [23]:
# 1.Count the number of books published after January 1st, 2000.
    
books_before_date = '''
SELECT 
    COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''


In [32]:
sql_results(books_before_date)

,books_after_2000
0,819


,books_after_2000
0,819


In [25]:
# 2️. Find the number of reviews and the average rating for each book

book_reviews_ratings = '''
SELECT 
    b.book_id,

    b.title,
    COUNT(DISTINCT rv.review_id) AS num_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY avg_rating DESC;
'''


In [34]:
sql_results(book_reviews_ratings)

,book_id,title,num_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


In [27]:
#3. Identify the publisher that released the largest number of books with more than 50 pages (to exclude short pamphlets).

publishers_pages = '''
SELECT 
    publishers.publisher, 
    COUNT(books.book_id) AS num_books
FROM books
JOIN publishers 
    ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC;
'''

In [35]:
pd.read_sql(publishers_pages, con=engine)

,publisher,num_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


In [36]:
# 4. Identify the author with the highest average book rating 
#     (consider only books with at least 50 ratings)

authors_avg_rating = '''
SELECT 
    authors.author,
    ROUND(AVG(ratings.rating), 2) AS avg_rating,
    COUNT(ratings.rating_id) AS total_ratings
FROM books
JOIN authors 
    ON books.author_id = authors.author_id
JOIN ratings 
    ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating_id) >= 50
ORDER BY avg_rating DESC;
'''

In [37]:
pd.read_sql(authors_avg_rating, con=engine)

,author,avg_rating,total_ratings
0,Diana Gabaldon,4.30,50
1,J.K. Rowling/Mary GrandPré,4.29,312
2,Agatha Christie,4.28,53
3,Markus Zusak/Cao Xuân Việt Khương,4.26,53
4,J.R.R. Tolkien,4.24,166
5,Roald Dahl/Quentin Blake,4.21,62
6,Louisa May Alcott,4.20,54
7,Rick Riordan,4.13,84
8,Arthur Golden,4.11,56
9,Stephen King,4.01,106


In [38]:
# 5. Find the average number of reviews among users who rated more than 50 books

avg_reviews_active_users = '''
WITH active_users AS (
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
reviews_count AS (
    SELECT 
        au.username,
        COUNT(rv.review_id) AS num_reviews
    FROM active_users AS au
    LEFT JOIN reviews AS rv ON rv.username = au.username
    GROUP BY au.username
)
SELECT 
    ROUND(AVG(num_reviews), 2) AS avg_reviews_per_active_user,
    COUNT(*) AS total_users
FROM reviews_count;
'''

In [39]:
pd.read_sql(avg_reviews_active_users, con=engine)

,avg_reviews_per_active_user,total_users
0,24.33,6


##  Conclusion

Out of 1,000 books in the dataset, **819** were published after the year 2000, showing a strong focus on modern titles.

**Penguin Books** is the top publisher, releasing the largest number of books with more than 50 pages (**42 titles**).  
**J.K. Rowling/Mary GrandPré** achieved the highest average rating (**4.29**), confirming strong reader preference.

Highly active users — those who rated more than 50 books — wrote an average of **24 reviews each**, indicating strong engagement among the platform’s core audience.

Overall, the data suggests a modern catalog dominated by major publishers and highly rated authors, supported by an active and loyal reader community.
